# Monte Carlo World Cup Simulation

### Elo Data
###### Source date: 05/22/2018 
https://www.eloratings.net/

In [65]:
participant_elos = {'Australia':1714, 
                    'Iran':1796,
                    'Japan':1692, 
                    'Saudi Arabia': 1600,
                    'South Korea':1745, 
                    'Egypt':1646,
                    'Morocco':1711,
                    'Nigeria':1699,
                    'Senegal':1747,
                    'Tunisia':1649,
                    'Costa Rica':1745,
                    'Mexico':1859,
                    'Panama':1669,
                    'Argentina': 1985,
                    'Brazil':2131,
                    'Colombia':1935,
                    'Peru':1906,
                    'Uruguay':1891,
                    'Belgium':1931,
                    'Croatia':1853,
                    'Denmark':1843,
                    'England':1941,
                    'France':1984,
                    'Germany':2092,
                    'Iceland':1787,
                    'Poland':1831,
                    'Portugal':1975,
                    'Russia':1685,       # Host Country
                    'Serbia':1770,
                    'Spain':2049,
                    'Sweden':1796,
                    'Switzerland':1879              
                   }

In [66]:
groups = {'Group_A': ['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay'],
          'Group_B': ['Portugal', 'Spain', 'Morocco', 'Iran'],
          'Group_C': ['France', 'Australia', 'Peru', 'Denmark'],
          'Group_D': ['Argentina', 'Iceland', 'Croatia', 'Nigeria'],
          'Group_E': ['Brazil', 'Switzerland', 'Costa Rica', 'Serbia'],
          'Group_F': ['Germany', 'Mexico', 'Sweden', 'South Korea'],
          'Group_G': ['Belgium', 'Panama', 'Tunisia', 'England'],
          'Group_H': ['Poland', 'Senegal', 'Colombia', 'Japan']       
        }

#### Win probability formula
$$W_e = \frac{1}{10^{\frac{-d_r}{400}} + 1}$$


In [67]:
from itertools import combinations
import random

def binomial_trial(success_rate):
    if success_rate > random.random():
        return True
    return False

def win_probability(team, opponent, elo=participant_elos):
    team_elo = elo[team]
    opponent_elo = elo[opponent]
    return (1 / (10 ** ( - ( team_elo-opponent_elo) / 400) + 1 ) )

def stochastic_game_winner(team, opponent, elo=participant_elos):
    team_win_rate = win_probability(team, opponent, elo)
    if binomial_trial(team_win_rate):
        return team
    
    return opponent

def single_group_run(group_name):
    group_members = groups[group_name]
    final_scores = {k:0 for k in group_members}
    for game in combinations(group_members, 2):
        win_prob = win_probability(game[0], game[1], participant_elos)
        if binomial_trial(win_prob):
            final_scores[game[0]] += 1
        else:
            final_scores[game[1]] += 1
            
    rank = sorted(final_scores, key=lambda k: final_scores[k], reverse=True)
    return rank[0], rank[1]

##OBSOLETE
def calculate_group_outcome_probabilities(group_name, number_of_runs):
    group_members = groups[group_name]
    first_place_count = {k:0 for k in group_members}
    second_place_count = {k:0 for k in group_members}
    for run in range(number_of_runs):
        first, second = single_group_run(group_name)
        first_place_count[first] += 1
        second_place_count[second] += 1
        
    first_place_normalized = {key:float(value)/sum(first_place_count.values()) for (key,value) in first_place_count.items()}
    second_place_normalized = {key:float(value)/sum(second_place_count.values()) for (key,value) in second_place_count.items()}
        
    return first_place_normalized, second_place_normalized

###### 8ths: Winner C vs Second D | Winner A vs Second B | Winner B vs Second A | Winner D vs Second C | Winner E vs Second F | Winner G vs Second E | Winner F vs Second E | Winner H vs Second G

###### quarter-finals: 

![image.png](img/match_paths.png)


In [97]:
def world_cup_single_run():
    a_first, a_second = single_group_run('Group_A')
    b_first, b_second = single_group_run('Group_B')
    c_first, c_second = single_group_run('Group_C')
    d_first, d_second = single_group_run('Group_D')
    e_first, e_second = single_group_run('Group_E')
    f_first, f_second = single_group_run('Group_F')
    g_first, g_second = single_group_run('Group_G')
    h_first, h_second = single_group_run('Group_H')
    
    sixteen_1 = stochastic_game_winner(a_first, b_second)
    sixteen_2 = stochastic_game_winner(c_first, d_second)
    sixteen_3 = stochastic_game_winner(e_first, f_second)
    sixteen_4 = stochastic_game_winner(g_first, h_second)
    sixteen_5 = stochastic_game_winner(b_first, a_second)
    sixteen_6 = stochastic_game_winner(d_first, c_second)
    sixteen_7 = stochastic_game_winner(f_first, e_second)
    sixteen_8 = stochastic_game_winner(h_first, g_second)
    
    quarter_final_1 = stochastic_game_winner(sixteen_1, sixteen_2)
    quarter_final_2 = stochastic_game_winner(sixteen_3, sixteen_4)
    quarter_final_3 = stochastic_game_winner(sixteen_5, sixteen_6)
    quarter_final_4 = stochastic_game_winner(sixteen_7, sixteen_8)
    
    semi_final_1 = stochastic_game_winner(quarter_final_1, quarter_final_2)
    semi_final_2 = stochastic_game_winner(quarter_final_3, quarter_final_4)
    
    winner = stochastic_game_winner(semi_final_1, semi_final_2)
    
    detailed_results = {'Second_Group_Stage': [a_second, b_second, c_second, d_second, e_second, f_second, g_second, h_second],
                        'Winner_Group_Stage': [a_first, b_first, c_first, d_first, e_first, f_first, g_first, h_first],
                        'Quater-Finalist': [sixteen_1, sixteen_2, sixteen_3, sixteen_4, sixteen_5, sixteen_6, sixteen_7, sixteen_8],
                        'Semi-Finalist': [quarter_final_1, quarter_final_2, quarter_final_3, quarter_final_4],
                        'Finalist': [semi_final_1, semi_final_2],
                        'Winner': [winner]
                       }
    
    
    return detailed_results

def calculate_world_cup_outcome_probabilities(number_of_runs):
    countries = participant_elos.keys()
    wins_total = {k:0 for k in countries}
    all_stages_count = {k:{k2:0 for k2 in countries} for k in ['Second_Group_Stage', 'Winner_Group_Stage', 'Quater-Finalist', 'Semi-Finalist', 'Finalist', 'Winner']}
    
    for run in range(number_of_runs):
        detailed_results = world_cup_single_run()
        for stage in detailed_results:
            for country in detailed_results[stage]:
                all_stages_count
    
    normalized_d = {k: v/number_of_runs for k,v in wins_total.items()}
    
    sorted_d = sorted(normalized_d.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_d
    
calculate_world_cup_outcome_probabilities(100)
    

[('Australia', 0.0),
 ('Iran', 0.0),
 ('Japan', 0.0),
 ('Saudi Arabia', 0.0),
 ('South Korea', 0.0),
 ('Egypt', 0.0),
 ('Morocco', 0.0),
 ('Nigeria', 0.0),
 ('Senegal', 0.0),
 ('Tunisia', 0.0),
 ('Costa Rica', 0.0),
 ('Mexico', 0.0),
 ('Panama', 0.0),
 ('Argentina', 0.0),
 ('Brazil', 0.0),
 ('Colombia', 0.0),
 ('Peru', 0.0),
 ('Uruguay', 0.0),
 ('Belgium', 0.0),
 ('Croatia', 0.0),
 ('Denmark', 0.0),
 ('England', 0.0),
 ('France', 0.0),
 ('Germany', 0.0),
 ('Iceland', 0.0),
 ('Poland', 0.0),
 ('Portugal', 0.0),
 ('Russia', 0.0),
 ('Serbia', 0.0),
 ('Spain', 0.0),
 ('Sweden', 0.0),
 ('Switzerland', 0.0)]

In [107]:
import pandas as pd

columns = ['Second_Group_Stage', 'Winner_Group_Stage', 'Quater-Finalist', 'Semi-Finalist', 'Finalist', 'Winner']
rows = list(participant_elos.keys())

df = pd.DataFrame(rows)
for column in columns:
    df[column] = 0
    
df = df.set_index(0)
df

,Second_Group_Stage,Winner_Group_Stage,Quater-Finalist,Semi-Finalist,Finalist,Winner
0,,,,,,
Australia,0,0,0,0,0,0
Iran,0,0,0,0,0,0
Japan,0,0,0,0,0,0
Saudi Arabia,0,0,0,0,0,0
South Korea,0,0,0,0,0,0
Egypt,0,0,0,0,0,0
Morocco,0,0,0,0,0,0
Nigeria,0,0,0,0,0,0
Senegal,0,0,0,0,0,0
